In [6]:
from selenium import webdriver
import time
import pandas as pd
import os


from selenium.webdriver.support.select import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

In [17]:
url ="https://www.linkedin.com/jobs/search/?currentJobId=3724435212&geoId=102787409&keywords=data%20scientist&location=Maroc&origin=JOB_SEARCH_PAGE_SEARCH_BUTTON&refresh=true"             


In [8]:
from deta import Deta
import pandas as pd
from datetime import datetime

class DataManager:
    def __init__(self, project_key, base_name):
        self.deta = Deta(project_key)
        self.db = self.deta.Base(base_name)

    def save_to_database(self, data_frame, table_name):
        # Charger les données existantes depuis la base de données
        existing_jobs = [item['ID_job'] for item in self.db.fetch().items]

        # Vérifier les doublons dans le DataFrame
        data_frame = data_frame[~data_frame['ID_job'].isin(existing_jobs)]

        # Itérer à travers chaque ligne du DataFrame
        for index, row in data_frame.iterrows():
#             # Convertir la date en format string avant l'insertion
#             row['datePublish'] = row['datePublish'].strftime('%Y-%m-%d %H:%M:%S')
            date_du_jour = datetime.now().strftime("%d-%m-%Y")
            # Insérer la ligne dans la base de données
            try:
                self.db.put({'ID_job': row['ID_job'], 'companyName': row['companyName'],
                             'jobTitles': row['jobTitles'], 'jobLinks': row['jobLinks'],
                             'datePublish': row['datePublish'],'date_insertion':date_du_jour,'sent':0})
            except Exception as e:
                # Gérer les erreurs d'insertion
                print(f"Error inserting row {index + 1}: {str(e)}")

        print("Data inserted successfully!")


In [9]:
import re
from selenium.common.exceptions import NoSuchElementException
import schedule
import time

class LinkedinScrapping:
    def __init__(self,date,field):
        self.date=date
        self.field=field
        self.driver = None
    def prepareEnv(self, url):
        self.driver = webdriver.Chrome()
        self.driver.get(url)
    def getJobCount(self,url):
        self.prepareEnv(url)  # Assurez-vous d'appeler la méthode pour initialiser le navigateur
        wait = WebDriverWait(self.driver, 10)
        job_count_element = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "results-context-header__job-count")))

        # Get the text from the job count element
        job_count_text = job_count_element.text
        numeric_value = re.sub(r'[^\d]+', '', job_count_text)
        y = pd.to_numeric(numeric_value)

        return y
    def exploreJobs(self,url):
        driver = self.driver
        max_failures = 3  # Set the maximum consecutive failures allowed
        failures = 0

        i = 0
        while i <= 30:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            i += 1

            try:
                # Find the 'See more jobs' button using XPath
                x = driver.find_element(By.XPATH, "//button[@aria-label='See more jobs']")
                driver.execute_script("arguments[0].click();", x)
                time.sleep(5)
                failures = 0  # Reset the failure counter if successful
                
            except NoSuchElementException:
                failures += 1
                if failures >= max_failures:
                    print("No more 'See more jobs' button found.")
                    break
        self.driver=driver
#___________________________________________________
    def getElements(self, url):
        
            y = self.getJobCount(url)
            self.exploreJobs(url)
            driver=self.driver
#             print(y)
            companynames = []
            job_titles = []
            job_links = []
            date_publish = []

            # Find elements using By.CLASS_NAME
            company_elements = driver.find_elements(By.CLASS_NAME, "base-search-card__subtitle")
            job_elements = driver.find_elements(By.CLASS_NAME, "base-search-card__title")
            link_elements = driver.find_elements(By.CLASS_NAME, "base-card__full-link")
            date_elements = driver.find_elements(By.CLASS_NAME, "job-search-card__listdate")

            for j in range(y):
                # Check if there are enough elements in each list
                if (
                    j < len(company_elements)
                    and j < len(job_elements)
                    and j < len(link_elements)
                    and j < len(date_elements)
                ):
                    # Extract the text from the elements
                    company = company_elements[j].text
                    job_title = job_elements[j].text
                    link = link_elements[j].get_attribute("href")
                    date_publ = date_elements[j].text

                    # Append the extracted data to the respective lists
                    companynames.append(company)
                    job_titles.append(job_title)
                    job_links.append(link)
                    date_publish.append(date_publ)
                else:
                    print(f"Insufficient elements at index {j}. Skipping.")

            # Return a dictionary with the extracted data
            return {
                "companyName": companynames,
                "jobTitles": job_titles,
                "jobLinks": job_links,
                "datePublish": date_publish
            }
    def convert_to_days(self, duration_string):
        # Fonction pour convertir une chaîne de durée en jours
        if "week" in duration_string:
            weeks = int(re.search(r'\d+', duration_string).group())
            return weeks * 7
        elif "month" in duration_string:
            months = int(re.search(r'\d+', duration_string).group())
            return months * 30  # Assuming 1 month = 30 days (adjust as needed)
        elif "day" in duration_string:
            return int(re.search(r'\d+', duration_string).group())
        elif "year" in duration_string:
            year = int(re.search(r'\d+', duration_string).group())
            return year * 365
        else:
            return 0
    def getPandasData(self,data):
            data=pd.DataFrame(data)
            data["datePublish"] = data["datePublish"].apply(self.convert_to_days)
            # Combinez "datePublish" et "jobTitles" pour former une clé primaire
            data["ID_job"] = data["datePublish"].astype(str) + '_' + data["jobTitles"]
            print(data)
            # Assurez-vous que la nouvelle colonne "primary_key" ne contient pas de doublons
            if data["ID_job"].duplicated().any():
                print("Attention: Des doublons dans la clé primaire détectés!")

            return data
    def save_to_database(self, data_frame):
            project_key = "a0zj6y7ndsx_ucmGqBMMLJu9PHPPSLtrpxKdg6Qj8J9Y"  # Remplacez par votre clé de projet Deta
            base_name = "jobs"      # Remplacez par le nom de votre base Deta

            data_manager = DataManager(project_key, base_name)
            data_manager.save_to_database(data_frame, table_name="jobs")
            print("\n data saved \n")
            
    def scrape_and_save(self, url):
            data = self.getElements(url)
            pandas_data = self.getPandasData(data)
            print(pandas_data)
            self.save_to_database(pandas_data)

    def schedule_scraping(self, url):
            # Planifier le scraping toutes les heures
#             schedule.every().hour.do(self.scrape_and_save, url, db_host, db_user, db_password, db_name, table_name)
            schedule.every(10).minutes.do(self.scrape_and_save, url)
            # Boucle pour exécuter la planification en continu
            while True:
                schedule.run_pending()
                time.sleep(1)


In [ ]:

linkedin_scraper = LinkedinScrapping(date="your_date" , field="your_field")
linkedin_scraper.schedule_scraping(url=url)


